# LLM-as-Judge
LangSmith 에서 제공되는 Off-the-shelf Evaluators 를 활용한다.

Off-the-shelf Evaluators 는 사전에 정의된 프롬프트 기반의 LLM 평가자를 의미한다.

Off-the-shelf Evaluators란 langsmith에서 만든 상용 evaluator이다. 한가지 제약이 있는데 `input`, `prediction`, `reference`가 필요하다.

쉽게 사용할 수 있는 이점이 있지만, 더 확장된 기능을 사용하기 위해서는 직접 평가자를 정의해야 한다.


기본적으로 다음의 3가지 정보를 LLM Evaluator 에 전달하여 평가를 진행한다.

**즉 llm으로 질문에대한 답변도 만들고, 평가도 하는것이다.**

- `input`: 질문. 보통 데이터셋의 Question 이 사용됩니다.
- `prediction`: LLM 이 생성한 답변. 보통 모델의 답변이 사용됩니다.
- `reference`: 정답 답변, Context 등 변칙적으로 활용이 가능.

참고 - https://docs.smith.langchain.com/evaluation/faq/evaluator-implementations

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


In [3]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

rag = PDFRAG(
    "./data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatOpenAI(model="gpt-4o-mini")
)

retriever = rag.create_retriever()

chain = rag.create_chain(retriever)

chain.invoke("삼성전자가 자체 개발한 생성형 AI이름은?")

"삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."

`ask_question` 이라는 이름으로 함수를 생성합니다. 입력으로는 `inputs` 라는 딕셔너리를 받고, 출력으로는 `answer` 라는 딕셔너리를 반환합니다.

In [4]:
# 질문에 대한 답변을 하는 함수 생성
def ask_question(inputs:dict):
    return {"answer": chain.invoke(inputs["question"])}

In [5]:
llm_answer = ask_question({"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"})
llm_answer

{'answer': "삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."}

In [6]:
# evaluator prompt출력을 위한 함수
def print_evaluator_prompt(evaluator):
    return evaluator.evaluator.prompt.pretty_print()

# Question-Answer Evaluator
가장 기본 기능을 가진 Evaluator 입니다. 질문(Query) 와 답변(Answer) 을 평가합니다.

사용자 입력은 input 으로 LLM 이 생성한 답변은 prediction 으로 정답 답변은 reference 로 정의됩니다.

(하지만, Prompt 변수는 `query`, `result`, `answer` 로 정의됩니다.)

- `query`: 질문
- `result`: LLM 답변
- `answer`: 정답 답변

In [10]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# qa 평가자 생성 : 질문과 답변이 연관이 있는지
qa_evaluator = LangChainStringEvaluator("qa")

# 프롬프트 출력 : (IPR)Input, Prediction, Reference가 없다. 이것들은 알아서 {query}, {result}, {answer}변환해서 넣어준다.
print_evaluator_prompt(qa_evaluator)

You are a teacher grading a quiz.
You are given a question, the student's answer, and the true answer, and are asked to score the student answer as either CORRECT or INCORRECT.

Example Format:
QUESTION: question here
STUDENT ANSWER: student's answer here
TRUE ANSWER: true answer here
GRADE: CORRECT or INCORRECT here

Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 

QUESTION: {query}
STUDENT ANSWER: {result}
TRUE ANSWER: {answer}
GRADE:


In [12]:
# 평가를 진행하고 출력된 URL로 이동하여 결과를 확인. 잘못되면 incorrect로 나온다.
# 질문과 답변을 고려해서 무조건 정답만 보고나서 유사한지 오답을 주는것이 아니라, 질문에 대한 대답을 출실히 했는지를 통해서 correct와 incorrect를 준다.
dataset_name = "RAG_EVAL_DATASET_teddynote"

experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[qa_evaluator],
    experiment_prefix="RAG_EVAL_teddynote",
    metadata = {"variant": "QA Evaluator를 활용한 평가"}
)

View the evaluation results for experiment: 'RAG_EVAL_teddynote-a9e3839c' at:
https://smith.langchain.com/o/2d0ce887-3f7f-59af-8d5e-12c1371ef5d5/datasets/334d943e-3086-4d03-91ae-4b5145224ffc/compare?selectedSessions=213d42a4-db49-4881-b10b-de78f15a89b1




0it [00:00, ?it/s]

# Context 에 기반한 답변 Evaluator
- `LangChainStringEvaluator("context_qa")`: LLM 체인에 정확성을 판단하는 데 참조 "context" 를 사용하도록 지시합니다.
- `LangChainStringEvaluator("cot_qa")`: "cot_qa" 는 "context_qa" 평가자와 유사하지만, 최종 판결을 결정하기 전에 LLM 의 '추론'을 사용하도록 지시한다는 점에서 차이가 있습니다.

contextual accuracy는 reference output을 참조하지 않는다. 우리가 검색한 정보에서 얼마만큼 충실하게 답변을 했는지를 평가한다.
cot contextual accuracy도 마찬가지다.

참고

먼저, Context 를 반환하는 함수를 정의해야 합니다: c`ontext_answer_rag_answer`

그 다음, `LangChainStringEvaluator` 를 생성합니다. 생성시 prepare_data 를 통해 위에서 정의한 함수의 반환 값을 적절하게 매핑합니다.

세부사항

- `run`: LLM 이 생성한 결과 (`context`, `answer`, `input`)
- `example`: 데이터셋에 정의된 데이터입니다. (`question` 과 `answer`)
`LangChainStringEvaluator` 이 평가를 수행하기 위하여 다음의 3가지 정보가 필요합니다.

- `prediction`: LLM 이 생성한 답변
- `reference`: 데이터셋에 정의된 답변
- `input`: 데이터셋에 정의된 질문
하지만, LangChainStringEvaluator("context_qa") 는 reference 를 Context 로 사용하기 때문에 다음과 같이 정의합니다.

(참고) 아래는 context_qa 평가자를 활용하기 위하여 context, answer, question 을 반환하는 함수를 정의하였습니다.

In [13]:
# Context를 반환하는 RAG 결과 반환 함수
def context_answer_rag_answer(inputs: dict):
    context = retriever.invoke(inputs["question"])
    return{
        "context" : "\n".join([doc.page_content for doc in context]),
        "answer" : chain.invoke(inputs["question"]),
        "query" : inputs["question"]
    }

In [14]:
context_answer_rag_answer({"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"})

{'context': '▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10\n   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11\n   ▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망··········································· 12\nSPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n어시스턴트를 적용한 구글 픽셀(Pixel)과 경쟁할 것으로 예상\n☞ 출처 : 삼성전자, ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스’ 공개, 2023.11.08.\n삼성전

In [16]:
# cot_qa 평가자 생성
cot_qa_evaluator = LangChainStringEvaluator(
    "cot_qa",
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],  # LLM 이 생성한 답변. llm이 만들어낸 답변과 조회한것들을 넣어주는것이 run
        "reference": run.outputs["context"],  # Context
        "input": example.inputs["question"],  # 데이터셋의 질문. 우리가 만든 데이터 질문셋이 example
    },
)

# context_qa 평가자 생성
context_qa_evaluator = LangChainStringEvaluator(
    "context_qa",
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],  # LLM 이 생성한 답변. llm이 만들어낸 답변과 조회한것들을 넣어주는것이 run
        "reference": run.outputs["context"],  # Context
        "input": example.inputs["question"],  # 데이터셋의 질문. 우리가 만든 데이터 질문셋이 example
    },
)

# evaluator prompt 출력
print_evaluator_prompt(context_qa_evaluator)

You are a teacher grading a quiz.
You are given a question, the context the question is about, and the student's answer. You are asked to score the student's answer as either CORRECT or INCORRECT, based on the context.

Example Format:
QUESTION: question here
CONTEXT: context the question is about here
STUDENT ANSWER: student's answer here
GRADE: CORRECT or INCORRECT here

Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 

QUESTION: {query}
CONTEXT: {context}
STUDENT ANSWER: {result}
GRADE:


In [17]:
# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET_teddynote"

# 평가 실행
evaluate(
    context_answer_rag_answer,
    data=dataset_name,
    evaluators=[cot_qa_evaluator, context_qa_evaluator], #2개의;rodml vudrkwkfmf tkdyd
    experiment_prefix="RAG_EVAL",
    metadata={
        "variant": "COT_QA & Context_QA Evaluator 를 활용한 평가",
    },
)

View the evaluation results for experiment: 'RAG_EVAL-3890cd9c' at:
https://smith.langchain.com/o/2d0ce887-3f7f-59af-8d5e-12c1371ef5d5/datasets/334d943e-3086-4d03-91ae-4b5145224ffc/compare?selectedSessions=cb3838dd-1f3f-460d-bb2a-90df63296a88




0it [00:00, ?it/s]

<ExperimentResults RAG_EVAL-3890cd9c>


Criteria
기준값 참조 레이블(정답 답변)이 없거나 얻기 힘든 경우 "criteria" 또는 "score" 평가자를 사용하여 사용자 지정 기준 집합에 대해 실행을 평가할 수 있습니다.

이는 모델의 답변에 대한 높은 수준의 의미론적 측면을 모니터링 하려는 경우에 유용합니다.

LangChainStringEvaluator("criteria", config={ "criteria": 아래 중 하나의 criterion })

| 기준             | 설명                                              |
|------------------|---------------------------------------------------|
| conciseness      | 답변이 간결하고 간단한지 평가                      |
| relevance        | 답변이 질문과 관련이 있는지 평가                   |
| correctness      | 답변이 옳은지 평가                                |
| coherence        | 답변이 일관성이 있는지 평가                        |
| harmfulness      | 답변이 해롭거나 유해한지 평가                      |
| maliciousness    | 답변이 악의적이거나 악화시키는지 평가              |
| helpfulness      | 답변이 도움이 되는지 평가                          |
| controversiality | 답변이 논란이 되는지 평가                         |
| misogyny         | 답변이 여성을 비하하는지 평가                     |
| criminality      | 답변이 범죄를 촉진하는지 평가                      |


In [18]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# 평가자 설정
criteria_evaluator = [
    LangChainStringEvaluator("criteria", config={"criteria": "conciseness"}),
    LangChainStringEvaluator("criteria", config={"criteria": "misogyny"}),
    LangChainStringEvaluator("criteria", config={"criteria": "criminality"}),
]

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET_teddynote"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=criteria_evaluator,
    experiment_prefix="CRITERIA-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "criteria 를 활용한 평가",
    },
)

View the evaluation results for experiment: 'CRITERIA-EVAL-c6207281' at:
https://smith.langchain.com/o/2d0ce887-3f7f-59af-8d5e-12c1371ef5d5/datasets/334d943e-3086-4d03-91ae-4b5145224ffc/compare?selectedSessions=9f9c770d-4514-4c48-a250-42fb817e5425




0it [00:00, ?it/s]

# 정답이 존재하는 경우 Evaluator 활용(labeled_criteria)
정답이 존재하는 경우, LLM 이 생성한 답변과 정답 답변을 비교하여 평가가 가능합니다.

아래의 예시처럼 `reference` 에는 정답 답변을, `prediction` 에는 LLM 이 생성한 답변을 전달합니다.

이 처럼 별도의 설정은 `prepare_data` 를 통해 정의합니다.

또한, 답변 평가에 활용되는 LLM 은 config 의 llm 을 통해 정의합니다.

**relevance는  관련성 즉, 할루시네이션 유무를, helpfulness는 도움이 되는지, 유익한지, 우리가만든 대답이 llm이 만든 답변에 얼마만큼 유사한지 판단하는것이다.**

In [19]:
from langsmith.evaluation import LangChainStringEvaluator
from langchain_openai import ChatOpenAI

# labeled_criteria 평가자 생성
labeled_criteria_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm": ChatOpenAI(temperature=0.0, model="gpt-4o-mini"),
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": example.outputs["answer"],  # 정답 답변
        "input": example.inputs["question"],
    },
)

# evaluator prompt 출력
print_evaluator_prompt(labeled_criteria_evaluator)

You are assessing a submitted answer on a given task or input based on a set of criteria. Here is the data:
[BEGIN DATA]
***
[Input]: {input}
***
[Submission]: {output}
***
[Criteria]: helpfulness: Is this submission helpful to the user, taking into account the correct reference answer?
***
[Reference]: {reference}
***
[END DATA]
Does the submission meet the Criteria? First, write out in a step by step manner your reasoning about each criterion to be sure that your conclusion is correct. Avoid simply stating the correct answers at the outset. Then print only the single character "Y" or "N" (without quotes or punctuation) on its own line corresponding to the correct answer of whether the submission meets all criteria. At the end, repeat just the letter again by itself on a new line.


아래는 relevance 를 평가하는 예시입니다.

이번에는 prepare_data 를 통해 reference 를 context 로 전달합니다.

In [20]:
from langchain_openai import ChatOpenAI

relevance_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": "relevance",
        "llm": ChatOpenAI(temperature=0.0, model="gpt-4o-mini"),
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["context"],  # Context 를 전달
        "input": example.inputs["question"],
    },
)

print_evaluator_prompt(relevance_evaluator)

You are assessing a submitted answer on a given task or input based on a set of criteria. Here is the data:
[BEGIN DATA]
***
[Input]: {input}
***
[Submission]: {output}
***
[Criteria]: relevance: Is the submission referring to a real quote from the text?
***
[Reference]: {reference}
***
[END DATA]
Does the submission meet the Criteria? First, write out in a step by step manner your reasoning about each criterion to be sure that your conclusion is correct. Avoid simply stating the correct answers at the outset. Then print only the single character "Y" or "N" (without quotes or punctuation) on its own line corresponding to the correct answer of whether the submission meets all criteria. At the end, repeat just the letter again by itself on a new line.


In [22]:
from langsmith.evaluation import evaluate

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET_teddynote"
# relevance는  관련성 즉, 할루시네이션 유무를, helpfulness는 도움이 되는지, 유익한지, 우리가만든 대답이 llm이 만든 답변에 얼마만큼 유사한지 판단하는것이다.
# 평가 실행
experiment_results = evaluate(
    context_answer_rag_answer,
    data=dataset_name,
    evaluators=[labeled_criteria_evaluator, relevance_evaluator], 
    experiment_prefix="LABELED-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "labeled_criteria evaluator 활용한 평가",
    },
)

View the evaluation results for experiment: 'LABELED-EVAL-30ff414e' at:
https://smith.langchain.com/o/2d0ce887-3f7f-59af-8d5e-12c1371ef5d5/datasets/334d943e-3086-4d03-91ae-4b5145224ffc/compare?selectedSessions=d60ea086-c159-4096-a5c8-304a601151c2




0it [00:00, ?it/s]

# 사용자 정의 점수 Evaluator(labeled_score_string)
아래는 점수를 반환하는 평가자 생성 예시입니다. `normalize_by` 를 통해 점수를 정규화할 수 있습니다. 변환된 점수는 (0 ~ 1) 사이의 값으로 정규화됩니다.

아래의 `accuracy` 는 사용자가 임의로 정의한 기준입니다. 적합한 Prompt 를 정의하여 사용할 수 있습니다.

In [33]:
from langsmith.evaluation import LangChainStringEvaluator
# accuracy를 1점부터 100점사이로 평가해달라는 정확도
# 점수를 반환하는 평가자 생성
labeled_score_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "criteria": {
            "accuracy": "How accurate is this prediction compared to the reference on a scale of 1-10?"
        },
        "normalize_by": 10,
        "llm": ChatOpenAI(temperature=0.0, model="gpt-4o-mini"),
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    },
)

print_evaluator_prompt(labeled_score_evaluator)

================================ System Message ================================

You are a helpful assistant.

================================ Human Message =================================

[Instruction]
Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below. {criteria}[Ground truth]
{reference}
Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the response on a scale of 1 to 10 by strictly following this format: "[[rating]]", for example: "Rating: [[5]]".

[Question]
{input}

[The Start of Assistant's Answer]
{prediction}
[The End of Assistant's Answer]


In [34]:
from langsmith.evaluation import evaluate

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[labeled_score_evaluator],
    experiment_prefix="LABELED-SCORE-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "labeled_score 활용한 평가",
    },
)

View the evaluation results for experiment: 'LABELED-SCORE-EVAL-db25142e' at:
https://smith.langchain.com/o/2d0ce887-3f7f-59af-8d5e-12c1371ef5d5/datasets/334d943e-3086-4d03-91ae-4b5145224ffc/compare?selectedSessions=4186d8a6-7f8a-4bd7-94a4-1b54fedb98d7




0it [00:00, ?it/s]